In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

def pegar_notas(textos):
    nfs = []
    for i, texto in enumerate(textos):
        nfs.append(textos[i])

    for i, nf in enumerate(textos):
        nfs[i] = (nfs[i].split())

    for i in range(len(textos)):
        
        nfs[i] = int(nfs[i][2])

    return(nfs)

In [29]:
#Abrindo o Chrome e entrando no Bling
username = 'rafael@nmtembalagens'
password = '*********'
navegador = webdriver.Chrome()
navegador.get('https://www.bling.com.br/login')
navegador.find_element(By.ID, 'username').send_keys(username)
navegador.find_element(By.ID, 'senha').send_keys(password)
navegador.find_element(By.CLASS_NAME, 'button-primary-site').click()
while len(navegador.find_elements(By.XPATH, '//*[@id="step-0"]/div[3]/div/button')) == 0:
    time.sleep(1)
navegador.find_element(By.XPATH, '//*[@id="step-0"]/div[3]/div/button').click() #Aceitar pop up
navegador.get('https://www.bling.com.br/vendas.php#list') #Entrar nos pedidos de venda

In [30]:
#APLICANDO FILTROS
navegador.find_element(By.ID, 'open-filter').click() #Clicar no botão para abrir os filtros
navegador.find_element(By.ID, 'sit-10').click() #Selecionar em Digitação
navegador.find_element(By.ID, 'psqTransportador').send_keys('Braspress Transportes Urgentes Ltda')# Escrever Transportadora
navegador.find_element(By.XPATH, '//*[@id="filter-button-area"]/button').click()
time.sleep(2)
textos = []#textos da lista em digitação
escritas = navegador.find_elements(By.CLASS_NAME, 'bli ')
for i, escrita in enumerate(escritas):
    if 'NF' in escrita.get_attribute('title'):
        textos.append(escrita.get_attribute('title'))
    
nfs_braspress = pegar_notas(textos)
print(nfs_braspress)

#Abrindo site da Braspress
sem_emails = []# Lista de clientes sem emails
CNPJ = "44368824000194"
navegador1 = webdriver.Chrome()
navegador1.get('https://www.braspress.com/rastreie-sua-encomenda/') #Acessar Braspress
while len(navegador1.find_elements(By.XPATH, '/html/body/footer/div[6]/div[3]/div/div[3]/button')) == 0:
    time.sleep(1)
navegador1.find_element(By.XPATH, '/html/body/footer/div[6]/div[3]/div/div[3]/button').click() #Aceitar os cookies
boxes = navegador.find_elements(By.CLASS_NAME, 'checkbox-item')#Checkbox
botoes = navegador.find_elements(By.CLASS_NAME, 'btn-invisible')
#loop entre as notas
for i in range(len(nfs_braspress)):
    navegador1.get('https://www.braspress.com/rastreie-sua-encomenda/') #Acessar Braspress
    element = navegador1.find_element(By.ID, 'cnpj-tracking')
    element.clear()
    element.send_keys(CNPJ)
    navegador1.execute_script("arguments[0].value = '{0}';".format(CNPJ), element)
    navegador1.find_element(By.ID, 'pedido-tracking').send_keys(nfs_braspress[i]) #Digitar a nota fiscal
    navegador1.find_element(By.CLASS_NAME, 'fas').click() #Buscar resultado
        #Se encontrar rastreio selecionar no bling - Se não pular para a próxima
    navegador1.switch_to.frame(navegador1.find_element(By.ID, 'iframe-tracking'))
    if len(navegador1.find_elements(By.CLASS_NAME, 'table')) == 0:
        print('NF {} não encontrada'.format(nfs_braspress[i]))#Pular a checkbox do Bling
    else:
        print('NF {} Rastreio encontrado'.format(nfs_braspress[i]))#Marcar a checkbox do Bling
        boxes[i].click()
        time.sleep(0.5)
        botoes[i].click()#Clicar no hamburguer
        time.sleep(2)
        dropdown_menu = navegador.find_elements(By.CLASS_NAME, 'dropdown-menu')
        enviar_emails = dropdown_menu[i].find_elements(By.TAG_NAME, 'li')
        time.sleep(1)
        for k in range(len(enviar_emails)):
            if enviar_emails[k].text == 'Enviar rastreio por e-mail':
                enviar_emails[k].click()
                time.sleep(1)
                if navegador.find_element(By.ID, 'rastreioEmailDestinatario').get_attribute('Value') != '': 
                    navegador.find_element(By.XPATH, '/html/body/div[27]/div[2]/div/button[1]').click()#Enviar o email de rastreio
                    time.sleep(1)
                else:
                    sem_emails.append(navegador.find_element(By.ID, 'rastreioNomeDestinatario').get_attribute('Value'))
                navegador.find_element(By.XPATH, '/html/body/div[27]/div[1]/button').click()#Fechar a aba de rastreio
        
navegador1.quit()

#Mudando os Status dos itens selecioandos
navegador.find_element(By.XPATH, '//*[@id="container"]/div[3]/div[2]/div/div[1]/button[2]').click()#Clicando em mudar situação dos pedidos
time.sleep(1)
try:
    navegador.find_element(By.XPATH, '//*[@id="situacoes_vendas"]/div[4]/div/label').click() #Clicando em "Em Andamento"
    time.sleep(1)
    navegador.find_element(By.XPATH, '/html/body/div[27]/div[2]/div/button[1]').click() #Clicando em Ok
    time.sleep(2)
except:
    pass

#Fechando erro final
if len(navegador.find_elements(By.XPATH, '/html/body/div[27]/div[3]/div/button')) == 1:
    navegador.find_element(By.XPATH, '/html/body/div[27]/div[3]/div/button').click()

[1109]
NF 1109 não encontrada


In [31]:
#REUNIDAS LOG

navegador.find_element(By.ID, 'psqTransportador').clear()
navegador.find_element(By.ID, 'psqTransportador').send_keys('Emp Reunidas Paulista de Transportes Ltda')# Escrever Transportadora
time.sleep(1)
navegador.find_element(By.XPATH, '//*[@id="filter-button-area"]/button').click()
time.sleep(2)
textos = []#textos da lista em digitação
escritas = navegador.find_elements(By.CLASS_NAME, 'bli ')
for i, escrita in enumerate(escritas):
    if 'NF' in escrita.get_attribute('title'):
        textos.append(escrita.get_attribute('title'))
        
nfs_reunidas = pegar_notas(textos)
print(nfs_reunidas)

boxes = navegador.find_elements(By.CLASS_NAME, 'checkbox-item')#Checkbox
botoes = navegador.find_elements(By.CLASS_NAME, 'btn-invisible')

navegador1 = webdriver.Chrome()
for i in range(len(nfs_reunidas)):
    navegador1.get('https://reunidaslog.com.br/servicos/')#Entrando no site
    navegador1.find_element(By.XPATH, '//*[@id="main"]/div/div/div/section[2]/div/div/div[1]/div/div/div[2]/div/form/table/tbody/tr[2]/td[2]/input').send_keys('44368824000194')
    navegador1.find_element(By.XPATH, '//*[@id="main"]/div/div/div/section[2]/div/div/div[1]/div/div/div[2]/div/form/table/tbody/tr[1]/td[2]/textarea').send_keys(nfs_reunidas[i])
    navegador1.find_element(By.XPATH, '//*[@id="main"]/div/div/div/section[2]/div/div/div[1]/div/div/div[2]/div/form/table/tbody/tr[3]/td[2]/a').click()
    elem = navegador1.find_element(By.CLASS_NAME, 'titulo')
    if 'Informação não disponível' in elem.text:
        print('NF' + ' ' + str(nfs_reunidas[i]) + ' Não tem rastreio')
    else:
        print('NF' + ' ' + str(nfs_reunidas[i]) + ' tem rastreio')
        boxes[i].click()
        botoes[i].click()
        time.sleep(2)
        dropdown_menu = navegador.find_elements(By.CLASS_NAME, 'dropdown-menu')
        enviar_emails = dropdown_menu[i].find_elements(By.TAG_NAME, 'li')
        time.sleep(2)
        for k in range(len(enviar_emails)):
            if enviar_emails[k].text == 'Enviar rastreio por e-mail':
                enviar_emails[k].click()
                time.sleep(1)
                if navegador.find_element(By.ID, 'rastreioEmailDestinatario').get_attribute('Value') != '':
                    navegador.find_element(By.XPATH, '/html/body/div[27]/div[2]/div/button[1]').click()#Enviar o email de rastreio
                    time.sleep(1)
                else:
                    sem_emails.append(navegador.find_element(By.ID, 'rastreioNomeDestinatario').get_attribute('Value'))
        time.sleep(1)
        navegador.find_element(By.XPATH, '/html/body/div[27]/div[1]/button').click()#Fechar a aba de rastreio
        
navegador1.quit()

#Mudando os Status dos itens selecioandos
navegador.find_element(By.XPATH, '//*[@id="container"]/div[3]/div[2]/div/div[1]/button[2]').click()#Clicando em mudar situação dos pedidos
time.sleep(1)
try:
    navegador.find_element(By.XPATH, '//*[@id="situacoes_vendas"]/div[4]/div/label').click() #Clicando em "Em Andamento"
    time.sleep(1)
    navegador.find_element(By.XPATH, '/html/body/div[27]/div[2]/div/button[1]').click() #Clicando em Ok
    time.sleep(2)
except:
    pass
    
#Fechando erro final
if len(navegador.find_elements(By.XPATH, '/html/body/div[27]/div[3]/div/button')) == 1:
    navegador.find_element(By.XPATH, '/html/body/div[27]/div[3]/div/button').click()

[]


In [32]:
#JAMEF

navegador.find_element(By.ID, 'psqTransportador').clear()
navegador.find_element(By.ID, 'psqTransportador').send_keys('Jamef Transportes Eireli')# Escrever Transportadora
time.sleep(1)
navegador.find_element(By.XPATH, '//*[@id="filter-button-area"]/button').click()
time.sleep(2)
textos = []#textos da lista em digitação
escritas = navegador.find_elements(By.CLASS_NAME, 'bli ')
for i, escrita in enumerate(escritas):
    if 'NF' in escrita.get_attribute('title'):
        textos.append(escrita.get_attribute('title'))
        
nfs_jamef = pegar_notas(textos)
print(nfs_jamef)

boxes = navegador.find_elements(By.CLASS_NAME, 'checkbox-item')#Checkbox
botoes = navegador.find_elements(By.CLASS_NAME, 'btn-invisible')

navegador1 = webdriver.Chrome()

navegador1.get('https://www.jamef.com.br/')#Entrando no site
time.sleep(2)
navegador1.find_element(By.ID, 'onetrust-accept-btn-handler').click()#Aceitar cookies
    
for i in range(len(nfs_jamef)):
    navegador1.get('https://www.jamef.com.br/')#Entrando no site
    time.sleep(2)
    navegador1.find_element(By.XPATH, '//*[@id="acompanhamento-da-carga"]/div[2]/form/div/div[1]/div/input').send_keys(nfs_jamef[i])
    navegador1.find_element(By.XPATH, '//*[@id="acompanhamento-da-carga"]/div[2]/form/div/button').click()#clicar em buscar
    while len(navegador1.find_elements(By.XPATH, '//*[@id="acompanhamento-da-carga"]/div[2]/form/div/div[2]/div/input')) == 0:
        time.sleep(1)#Esperar carregar
    navegador1.find_element(By.XPATH, '//*[@id="acompanhamento-da-carga"]/div[2]/form/div/div[2]/div/input').send_keys('44368824000194')
    navegador1.find_element(By.XPATH, '//*[@id="acompanhamento-da-carga"]/div[2]/form/div/button').click()#clicar em buscar
    time.sleep(10)
    if len(navegador1.find_elements(By.XPATH, '/html/body/div[4]/div/div[1]/div[1]')) == 1: #Não tem rastreio
        print('NF' + ' ' + str(nfs_jamef[i]) + ' Não tem rastreio')
    else:
        print('NF' + ' ' + str(nfs_jamef[i]) + ' Tem rastreio')
        boxes[i].click()
        botoes[i].click()
        time.sleep(2)
        dropdown_menu = navegador.find_elements(By.CLASS_NAME, 'dropdown-menu')
        enviar_emails = dropdown_menu[i].find_elements(By.TAG_NAME, 'li')
        time.sleep(2)
        for k in range(len(enviar_emails)):
            if enviar_emails[k].text == 'Enviar rastreio por e-mail':
                enviar_emails[k].click()
                time.sleep(1)
                if navegador.find_element(By.ID, 'rastreioEmailDestinatario').get_attribute('Value') != '':
                    navegador.find_element(By.XPATH, '/html/body/div[27]/div[2]/div/button[1]').click()#Enviar o email de rastreio
                else:
                    sem_emails.append(navegador.find_element(By.ID, 'rastreioNomeDestinatario').get_attribute('Value'))
        time.sleep(1)
        navegador.find_element(By.XPATH, '/html/body/div[27]/div[1]/button').click()#Fechar a aba de rastreio

navegador1.quit()

#Mudando os Status dos itens selecioandos
navegador.find_element(By.XPATH, '//*[@id="container"]/div[3]/div[2]/div/div[1]/button[2]').click()#Clicando em mudar situação dos pedidos
time.sleep(1)
try:
    navegador.find_element(By.XPATH, '//*[@id="situacoes_vendas"]/div[4]/div/label').click() #Clicando em "Em Andamento"
    time.sleep(1)
    navegador.find_element(By.XPATH, '/html/body/div[27]/div[2]/div/button[1]').click() #Clicando em Ok
    time.sleep(2)
except:
    pass

#Fechando erro final
if len(navegador.find_elements(By.XPATH, '/html/body/div[27]/div[3]/div/button')) == 1:
    navegador.find_element(By.XPATH, '/html/body/div[27]/div[3]/div/button').click()

[1113, 1133]
NF 1113 Não tem rastreio
NF 1133 Não tem rastreio


In [33]:
#CORREIOS

navegador.find_element(By.ID, 'psqTransportador').clear()
time.sleep(1)
navegador.find_element(By.XPATH, '//*[@id="filter-button-area"]/button').click()
time.sleep(2)

boxes = navegador.find_elements(By.CLASS_NAME, 'checkbox-item')#Checkbox
botoes = navegador.find_elements(By.CLASS_NAME, 'btn-invisible')

lista = navegador.find_elements(By.CLASS_NAME, 'marcadores')
for i, item in enumerate(lista):
    if len(item.find_elements(By.TAG_NAME, 'img')) != 0:
        imagem = item.find_element(By.TAG_NAME, 'img')
        if 'correios' in imagem.get_attribute('src'):#Pegando os itens do correio
            imagem.click()
            time.sleep(1)
            if len(navegador.find_elements(By.XPATH, '//*[@id="mensagemRastro"]/div/p')) == 1: #Não tem rastreio
                navegador.find_element(By.XPATH, '/html/body/div[27]/div[1]/button').click()#Fechar a aba de rastreio
                print(i, 'não tem rastreio')
            else:
                navegador.find_element(By.XPATH, '//*[@id="enviarEmailRastro"]').click()#Evniar email de rastreio
                time.sleep(1)
                navegador.find_element(By.XPATH, '/html/body/div[27]/div[1]/button').click()#Fechar aba dos correios
                time.sleep(1)
                boxes[i].click()
                time.sleep(0.5)
                time.sleep(1)
            
            
#Mudando os Status dos itens selecioandos
navegador.find_element(By.XPATH, '//*[@id="container"]/div[3]/div[2]/div/div[1]/button[2]').click()#Clicando em mudar situação dos pedidos
time.sleep(1)
try:
    navegador.find_element(By.XPATH, '//*[@id="situacoes_vendas"]/div[4]/div/label').click() #Clicando em "Em Andamento"
    time.sleep(1)
    navegador.find_element(By.XPATH, '/html/body/div[27]/div[2]/div/button[1]').click() #Clicando em Ok
    time.sleep(2)
except:
    pass

#Fechando erro final
if len(navegador.find_elements(By.XPATH, '/html/body/div[27]/div[3]/div/button')) == 1:
    navegador.find_element(By.XPATH, '/html/body/div[27]/div[3]/div/button').click()

In [ ]:
# Checando se mercadoria foi entregue
#Braspress 1
#navegador.find_element(By.ID, 'open-filter').click() #Clicar no botão para abrir os filtros
navegador.find_element(By.ID, 'sit-3').click() #Selecionar em Andamaneto
navegador.find_element(By.ID, 'psqTransportador').clear()
navegador.find_element(By.ID, 'psqTransportador').send_keys('Braspress Transportes Urgentes Ltda')# Escrever Transportadora
navegador.find_element(By.XPATH, '//*[@id="filter-button-area"]/button').click()#Clicar em filtrar
time.sleep(2)
textos = []#textos da lista em digitação
escritas = navegador.find_elements(By.CLASS_NAME, 'bli ')
for i, escrita in enumerate(escritas):
    if 'NF' in escrita.get_attribute('title'):
        textos.append(escrita.get_attribute('title'))
    
nfs_braspress = pegar_notas(textos)

#Braspress 2

emails_braspress = []
CNPJ = "44368824000194"
navegador1 = webdriver.Chrome()
navegador1.get('https://www.braspress.com/rastreie-sua-encomenda/') #Acessar Braspress
while len(navegador1.find_elements(By.XPATH, '/html/body/footer/div[6]/div[3]/div/div[3]/button')) == 0:
    time.sleep(1)
navegador1.find_element(By.XPATH, '/html/body/footer/div[6]/div[3]/div/div[3]/button').click() #Aceitar os cookies
boxes = navegador.find_elements(By.CLASS_NAME, 'checkbox-item')#Checkbox
botoes = navegador.find_elements(By.CLASS_NAME, 'btn-invisible')#Botão hamburguer

#loop entre as notas
for i in range(len(nfs_braspress)):
    navegador1.get('https://www.braspress.com/rastreie-sua-encomenda/') #Acessar Braspress
    element = navegador1.find_element(By.ID, 'cnpj-tracking')
    element.clear()
    element.send_keys(CNPJ)
    navegador1.execute_script("arguments[0].value = '{0}';".format(CNPJ), element)
    navegador1.find_element(By.ID, 'pedido-tracking').send_keys(nfs_braspress[i]) #Digitar a nota fiscal
    navegador1.find_element(By.CLASS_NAME, 'fas').click() #Buscar resultado
    #Checar se o pedido foi entregue
    navegador1.switch_to.frame(navegador1.find_element(By.ID, 'iframe-tracking'))
    colunas = navegador1.find_elements(By.TAG_NAME, 'td')
    if colunas[4].text == 'Entrega Realizada': #Encomenda entregue
        boxes[i].click()
        time.sleep(1)
        botoes[i].click()
        time.sleep(1)
        dropdown_menu = navegador.find_elements(By.CLASS_NAME, 'dropdown-menu')
        enviar_emails = dropdown_menu[i].find_elements(By.TAG_NAME, 'li')
        time.sleep(2)
        for k in range(len(enviar_emails)):
            if enviar_emails[k].text == 'Enviar rastreio por e-mail':
                enviar_emails[k].click()#Clicar em enviar email
                time.sleep(1)
                emails_braspress.append(navegador.find_element(By.ID, 'rastreioEmailDestinatario').get_attribute('Value'))
        time.sleep(1)
        navegador.find_element(By.XPATH, '/html/body/div[27]/div[1]/button').click()#Fechar a aba de rastreio
        print('NF' + str(nfs_braspress[i]) + ' Foi entregue')
    else:#Mercadoria em andamento ainda
        print('NF' + str(nfs_braspress[i]) + ' Ainda não foi entregue')
        
print(emails_braspress)
        
#Mudando Status para finalizado    
navegador.find_element(By.XPATH, '//*[@id="container"]/div[3]/div[2]/div/div[1]/button[2]').click()#Clicando em mudar situação dos pedidos
time.sleep(1)
try:
    navegador.find_element(By.XPATH, '//*[@id="situacoes_vendas"]/div[9]/div/label').click()#Clicando em finalizado
    time.sleep(1)
    navegador.find_element(By.XPATH, '/html/body/div[27]/div[2]/div/button[1]').click() #Clicando em Ok
    time.sleep(1)
except:
    pass

navegador1.quit()

In [35]:
# Checando se mercadoria foi entregue
#Reunidas Log

emails_reunidas = []
navegador.find_element(By.ID, 'psqTransportador').clear()
navegador.find_element(By.ID, 'psqTransportador').send_keys('Emp Reunidas Paulista de Transportes Ltda')# Escrever Transportadora
time.sleep(1)
navegador.find_element(By.XPATH, '//*[@id="filter-button-area"]/button').click()
time.sleep(2)
textos = []#textos da lista em digitação
escritas = navegador.find_elements(By.CLASS_NAME, 'bli ')
for i, escrita in enumerate(escritas):
    if 'NF' in escrita.get_attribute('title'):
        textos.append(escrita.get_attribute('title'))
        
nfs_reunidas = pegar_notas(textos)
print(nfs_reunidas)

boxes = navegador.find_elements(By.CLASS_NAME, 'checkbox-item')#Checkbox
botoes = navegador.find_elements(By.CLASS_NAME, 'btn-invisible')#Botão hamburguer

navegador1 = webdriver.Chrome()
for i in range(len(nfs_reunidas)):
    navegador1.get('https://reunidaslog.com.br/servicos/')#Entrando no site
    navegador1.find_element(By.XPATH, '//*[@id="main"]/div/div/div/section[2]/div/div/div[1]/div/div/div[2]/div/form/table/tbody/tr[2]/td[2]/input').send_keys('44368824000194')
    navegador1.find_element(By.XPATH, '//*[@id="main"]/div/div/div/section[2]/div/div/div[1]/div/div/div[2]/div/form/table/tbody/tr[1]/td[2]/textarea').send_keys(nfs_reunidas[i])
    navegador1.find_element(By.XPATH, '//*[@id="main"]/div/div/div/section[2]/div/div/div[1]/div/div/div[2]/div/form/table/tbody/tr[3]/td[2]/a').click()
    elem = navegador1.find_element(By.CLASS_NAME, 'titulo')
    if 'MERCADORIA ENTREGU' in elem.text: #Mercadoria entregue
        boxes[i].click()
        time.sleep(1)
        botoes[i].click()
        time.sleep(1)
        dropdown_menu = navegador.find_elements(By.CLASS_NAME, 'dropdown-menu')
        enviar_emails = dropdown_menu[i].find_elements(By.TAG_NAME, 'li')
        time.sleep(2)
        for k in range(len(enviar_emails)):
            if enviar_emails[k].text == 'Enviar rastreio por e-mail':
                enviar_emails[k].click()#Clicar em enviar email
                time.sleep(1)
                emails_reunidas.append(navegador.find_element(By.ID, 'rastreioEmailDestinatario').get_attribute('Value'))
        time.sleep(1)
        navegador.find_element(By.XPATH, '/html/body/div[27]/div[1]/button').click()#Fechar a aba de rastreio
        print('NF' + str(nfs_reunidas[i]) + ' Foi entregue')
    else:# Mercadoria em andamento ainda
        print('NF' + str(nfs_reunidas[i]) + ' Ainda não foi entregue')
        
print(emails_reunidas)

#Mudando Status para finalizado
navegador.find_element(By.XPATH, '//*[@id="container"]/div[3]/div[2]/div/div[1]/button[2]').click()#Clicando em mudar situação dos pedidos
time.sleep(1)
try:
    navegador.find_element(By.XPATH, '//*[@id="situacoes_vendas"]/div[9]/div/label').click()#Clicando em finalizado
    time.sleep(1)
    navegador.find_element(By.XPATH, '/html/body/div[27]/div[2]/div/button[1]').click() #Clicando em Ok
    time.sleep(1)
except:
    pass
        
navegador1.quit()

[1116, 1095, 1121, 1134, 1120]
NF1116 Foi entregue
NF1095 Foi entregue
NF1121 Ainda não foi entregue
NF1134 Ainda não foi entregue
NF1120 Ainda não foi entregue
['marco@vinilburger.com.br', 'chinasangastronomia@hotmail.com']


In [36]:
# Checando se mercadoria foi entregue
#Jamef

emails_jamef = []

navegador.find_element(By.ID, 'psqTransportador').clear()
navegador.find_element(By.ID, 'psqTransportador').send_keys('Jamef Transportes Eireli')# Escrever Transportadora
time.sleep(1)
navegador.find_element(By.XPATH, '//*[@id="filter-button-area"]/button').click()
time.sleep(2)
textos = []#textos da lista em digitação
escritas = navegador.find_elements(By.CLASS_NAME, 'bli ')
for i, escrita in enumerate(escritas):
    if 'NF' in escrita.get_attribute('title'):
        textos.append(escrita.get_attribute('title'))
        
nfs_jamef = pegar_notas(textos)
print(nfs_jamef)

boxes = navegador.find_elements(By.CLASS_NAME, 'checkbox-item')#Checkbox
botoes = navegador.find_elements(By.CLASS_NAME, 'btn-invisible')

navegador1 = webdriver.Chrome()

navegador1.get('https://www.jamef.com.br/')#Entrando no site
while len(navegador1.find_elements(By.ID, 'onetrust-accept-btn-handler')) == 0:
    time.sleep(1)
navegador1.find_element(By.ID, 'onetrust-accept-btn-handler').click()#Aceitar cookies
    
for i in range(len(nfs_jamef)):
    navegador1.get('https://www.jamef.com.br/')#Entrando no site
    time.sleep(2)
    navegador1.find_element(By.XPATH, '//*[@id="acompanhamento-da-carga"]/div[2]/form/div/div[1]/div/input').send_keys(nfs_jamef[i])
    navegador1.find_element(By.XPATH, '//*[@id="acompanhamento-da-carga"]/div[2]/form/div/button').click()#clicar em buscar
    while len(navegador1.find_elements(By.XPATH, '//*[@id="acompanhamento-da-carga"]/div[2]/form/div/div[2]/div/input')) == 0:
        time.sleep(1)#Esperar carregar
    navegador1.find_element(By.XPATH, '//*[@id="acompanhamento-da-carga"]/div[2]/form/div/div[2]/div/input').send_keys('44368824000194')
    navegador1.find_element(By.XPATH, '//*[@id="acompanhamento-da-carga"]/div[2]/form/div/button').click()#clicar em buscar
    time.sleep(12)
    # Checar se foi entregue
    status = navegador1.find_elements(By.CLASS_NAME, 'active')
    if len(status) == 6:#Encomenda entregada
        boxes[i].click()
        time.sleep(1)
        botoes[i].click()
        time.sleep(1)
        dropdown_menu = navegador.find_elements(By.CLASS_NAME, 'dropdown-menu')
        enviar_emails = dropdown_menu[i].find_elements(By.TAG_NAME, 'li')
        time.sleep(2)
        for k in range(len(enviar_emails)):
            if enviar_emails[k].text == 'Enviar rastreio por e-mail':
                enviar_emails[k].click()#Clicar em enviar email
                time.sleep(1)
                emails_jamef.append(navegador.find_element(By.ID, 'rastreioEmailDestinatario').get_attribute('Value'))
        time.sleep(1)
        navegador.find_element(By.XPATH, '/html/body/div[27]/div[1]/button').click()#Fechar a aba de rastreio
        print('NF' + str(nfs_jamef[i]) + ' foi entregue')
    else:#Encomenda ainda não foi entregue
        print('NF' + str(nfs_jamef[i]) + ' ainda não foi entregue')
        
print(emails_jamef)

#Mudando Status para finalizado
navegador.find_element(By.XPATH, '//*[@id="container"]/div[3]/div[2]/div/div[1]/button[2]').click()#Clicando em mudar situação dos pedidos
time.sleep(1)
try:
    navegador.find_element(By.XPATH, '//*[@id="situacoes_vendas"]/div[9]/div/label').click()#Clicando em finalizado
    time.sleep(1)
    navegador.find_element(By.XPATH, '/html/body/div[27]/div[2]/div/button[1]').click() #Clicando em Ok
    time.sleep(1)
except:
    pass


navegador1.quit()

[1114, 1107]
NF1114 ainda não foi entregue
NF1107 ainda não foi entregue
[]


In [37]:
# Checando se mercadoria foi entregue
#Correios

emails_correios = []

navegador.find_element(By.ID, 'psqTransportador').clear()
time.sleep(1)
navegador.find_element(By.XPATH, '//*[@id="filter-button-area"]/button').click()
time.sleep(2)

boxes = navegador.find_elements(By.CLASS_NAME, 'checkbox-item')#Checkbox
botoes = navegador.find_elements(By.CLASS_NAME, 'btn-invisible')

lista = navegador.find_elements(By.CLASS_NAME, 'marcadores')
for i, item in enumerate(lista):
    if len(item.find_elements(By.TAG_NAME, 'img')) != 0:
        imagem = item.find_element(By.TAG_NAME, 'img')
        if 'correios' in imagem.get_attribute('src'):#Pegando os itens do correio
            imagem.click()
            time.sleep(1)
            if navegador.find_element(By.ID, 'situacaoRastro').text == 'Objeto entregue ao destinatário':
                navegador.find_element(By.XPATH, '/html/body/div[27]/div[1]/button').click()#Fechar a aba de rastreio
                boxes[i].click()
                time.sleep(0.5)
                botoes[i].click()
                time.sleep(1)
                dropdown_menu = navegador.find_elements(By.CLASS_NAME, 'dropdown-menu')
                enviar_emails = dropdown_menu[i].find_elements(By.TAG_NAME, 'li')
                time.sleep(2)
                for k in range(len(enviar_emails)):
                    if enviar_emails[k].text == 'Enviar rastreio por e-mail':
                        enviar_emails[k].click()#Clicar em enviar email
                        time.sleep(1)
                        emails_correios.append(navegador.find_element(By.ID, 'rastreioEmailDestinatario').get_attribute('Value'))#Pegando email
                        navegador.find_element(By.XPATH, '/html/body/div[27]/div[1]/button').click()#Fechar a aba de rastreio
            else:
                navegador.find_element(By.XPATH, '/html/body/div[27]/div[1]/button').click()#Fechar a aba de rastreio
                
print(emails_correios)

#Mudando Status para finalizado
navegador.find_element(By.XPATH, '//*[@id="container"]/div[3]/div[2]/div/div[1]/button[2]').click()#Clicando em mudar situação dos pedidos
time.sleep(1)
try:
    navegador.find_element(By.XPATH, '//*[@id="situacoes_vendas"]/div[9]/div/label').click()#Clicando em finalizado
    time.sleep(1)
    navegador.find_element(By.XPATH, '/html/body/div[27]/div[2]/div/button[1]').click() #Clicando em Ok
    time.sleep(1)
except:
    pass

[]


In [38]:
#Enviar EMAILS - PESQUISA DE SATISFAÇÃO DO CLIENTE
import win32com.client as win32

emails = emails_braspress + emails_reunidas + emails_jamef + emails_correios #Junção dos emails dos clientes das diversas transportadoras
emails_teste = ['rafael_viott@hotmail.com', 'rafaelpviott@gmail.com']

for i in range(len(emails)):# Percorrendo todos os emails dos clientes
    outlook = win32.Dispatch("Outlook.Application")
    mail = outlook.CreateItem(0) #Criando o email, email principal do outlook
    mail.To = emails[i]#Email do cliente
    mail.Subject = 'Pesquisa de Satisfação Terpell Embalagens'#Assunto do email
    mail.Body = '''Olá, tudo bem? Vimos aqui que recebeu nosso produto, queremos saber qual foi sua experiência comprando com a gente! Preparamos algumas perguntas para obter um feedback para conseguir melhorar nosso atendimento e produto. Não leva nem 1 minutinho.

https://docs.google.com/forms/d/e/1FAIpQLSfusGJcuBpAY4MfpcQe6YY8_xmDSDJYHskWbv2qMlDGWECK0g/viewform

A Terpell Embalagens agradece!! =) ''' #Envio do Feedback
    mail.Send()#Enviar email
    print('email de Rastreio enviado para {} com sucesso!'.format(emails[i]))

email de Rastreio enviado para Fls.leiroze@gmail.com com sucesso!
email de Rastreio enviado para fornecedorescampeche@gmail.com com sucesso!
email de Rastreio enviado para cleriaphamada@hotmail.com com sucesso!
email de Rastreio enviado para marco@vinilburger.com.br com sucesso!
email de Rastreio enviado para chinasangastronomia@hotmail.com com sucesso!


In [16]:
emails = emails_braspress + emails_reunidas + emails_jamef + emails_correios #Junção dos emails dos clientes das diversas transportadoras
for i in range(len(emails)):
    print(emails[i])

fabinhocaldeira@hotmail.com
comercial@cafecreme.com.br
